In [12]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="openai/gpt-oss-20b",
    temperature=0.7
)

In [14]:
response = llm.invoke("Hello! Can you tell me a short joke in indian context about startup founders?")
print(response.content)

Why did the startup founder bring a chai pot to the investor meeting?

Because he heard the investors love a good “pitch” and a “chai‑t” of confidence!


### Search node

In [15]:
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=3)
web_search_results = web_search.invoke("who is the Chief Minister of bangalore?")

print(web_search_results["results"][0]["content"])

3 days ago—Bengaluru:Chief Minister Siddaramaiahon Saturday was unimpressed by the city's road condition and blamed rains for it as he went on an


In [16]:
from langgraph.prebuilt import create_react_agent

groq_llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
)

# 2. Create BetterMeals research agent using Groq model
research_agent = create_react_agent(
    model=groq_llm,
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent",
)


In [17]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [18]:
# for chunk in research_agent.stream(
#     {"messages": [{"role": "user", "content": "who is the Chief Minister of bangalore?"}]}
# ):
#     pretty_print_messages(chunk)

In [19]:
from typing import TypedDict, Optional

class State(TypedDict):
    raw_message: str
    query_type: Optional[str]
    response: Optional[str]


In [23]:
def handle_meal_doubt(state: State):
    """Answer nutrition/meal-related user doubts."""
    return {"response": f"[MealDoubtAgent] Answering: {state['raw_message']}"}

def handle_user_update(state: State):
    """Update user preferences or nutrition plan."""
    return {"response": f"[UserUpdateAgent] Updating plan with: {state['raw_message']}"}

def handle_cook_update(state: State):
    """Handle cook updates like missing ingredients."""
    return {"response": f"[CookUpdateAgent] Cook update received: {state['raw_message']}"}

def handle_grocery_order(state: State):
    """Trigger or update grocery orders."""
    return {"response": f"[GroceryOrderAgent] Ordered groceries: {state['raw_message']}"}


In [24]:
supervisor_agent = create_react_agent(
    model=groq_llm,
    tools=[
        handle_meal_doubt,
        handle_user_update,
        handle_cook_update,
        handle_grocery_order,
    ],
    prompt=(
        "You are the Supervisor Agent for BetterMeals.\n"
        "- Route meal/nutrition doubts to MealDoubtAgent.\n"
        "- Route user profile or preference changes to UserUpdateAgent.\n"
        "- Route cook updates (missing ingredients) to CookUpdateAgent.\n"
        "- Route grocery ordering requests to GroceryOrderAgent.\n"
        "Always assign tasks to the correct agent. Do not solve them yourself."
    ),
    name="supervisor",
)

In [25]:
for chunk in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": "order brocoli for dinner?"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  handle_grocery_order (fc_edd4c7e9-0432-4cc1-8493-daa031104f88)
 Call ID: fc_edd4c7e9-0432-4cc1-8493-daa031104f88
  Args:
    state: {'query_type': 'grocery_order', 'raw_message': 'order brocoli for dinner?', 'response': ''}


Update from node tools:


================================= Tool Message =================================
Name: handle_grocery_order

{"response": "[GroceryOrderAgent] Ordered groceries: order brocoli for dinner?"}


Update from node agent:


================================== Ai Message ==================================
Name: supervisor

[GroceryOrderAgent] Ordered groceries: order brocoli for dinner?


